In [ ]:
import numpy as np
import uproot

: 

In [61]:
fname = "/pnfs/sbnd/scratch/users/lynnt/miniprod_rockbox/12716802_239/output.flat.root"
# fname = "/exp/sbnd/data/users/gputnam/gump.df"
# fname = "/exp/sbnd/app/users/munjung/osc/sbnana/sbnana/SBNAna/osc-villiage/gump/test.df"
# fname = "/pnfs/icarus/scratch/users/mueller/mc_run2_collonly/flat/mc-v09_84_00_01-202403-cnaf-corrsce_9999_20240325T114754-GenGenieFilter_20240325T121332-G4_20240325T131523-DetSim_20240325T135149-MCstage0-MCstage1.mlreco.flat.root"

In [ ]:
def mag(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

def magdf(df):
    return mag(df.x, df.y, df.z)

def dmagdf(df1, df2):
    return mag(df1.x - df2.x, df1.y - df2.y, df1.z - df2.z)

In [6]:
import glob
import numpy as np
import uproot
import pandas as pd
from tqdm.auto import tqdm
import subprocess
from multiprocessing import Pool
import multiprocessing
import os
import dill
import sys
from functools import partial

CPU_COUNT = multiprocessing.cpu_count()

if CPU_COUNT == 0:
    CPU_COUNT = os.cpu_count()

class NTupleProc(object):
    def __init__(self, f=None, name="None"):
        self.name = name
        self.f = f

    def __call__(self, df):
        return self.f(df)

    def __bool__(self):
        return self.f is not None

    # make work with pickle for multiprocessing
    def __getstate__(self):
        return dill.dumps({"f": self.f, "name": self.name})

    def __setstate__(self, state):
        data = dill.loads(state)
        self.f = data["f"]
        self.name = data["name"]



def _loaddf(applyfs, g):
    # fname, index, applyfs = inp
    index, fname = g
    # Convert pnfs to xroot URL's
#    if fname.startswith("/pnfs"):
#        fname = fname.replace("/pnfs", "root://fndcadoor.fnal.gov:1094/pnfs/fnal.gov/usr")
#    # fix xroot URL's
#    elif fname.startswith("xroot"):
#        fname = fname[1:]

    madef = False

    # Flatten non-flat cafs
    #if "flat" not in fname.split("/")[-1].split("."):
    #    flatcaf = "/tmp/" + fname.split("/")[-1].split(".")[0] + ".flat.root"
    #    subprocess.run(["flatten_caf", fname, flatcaf],  stdout=subprocess.DEVNULL)
    #    fname = flatcaf
    #    madef = True
   
    try:
        f = uproot.open(fname, timeout=120)
    except (OSError, ValueError) as e:
        print("Could not open file (%s) due to exception. Skipping..." % fname) 
        print(e)
        return None
    with f:
        try:
            dfs = [applyf(f) for applyf in applyfs]
        except Exception as e:
            print("Error processing file (%s). Skipping..." % fname)
            print(e)
            return None

        # Set an index on the NTuple number to make sure we keep track of what is where
        for i in range(len(dfs)):
            if dfs[i] is not None:
                dfs[i]["__ntuple"] = index
                dfs[i].set_index("__ntuple", append=True, inplace=True)
                dfs[i] = dfs[i].reorder_levels([dfs[i].index.nlevels-1] + list(range(0, dfs[i].index.nlevels-1)))
            else:
                dfs[i] = []

    if madef:
        os.remove(fname)

    return dfs

class NTupleGlob(object):
    def __init__(self, g, branches):
        if isinstance(g, list) and len(g) == 1:
            g = g[0]
        if isinstance(g, list):
            self.glob = g
        elif g.endswith(".list"):
            with open(g) as f:
                self.glob = [line.rstrip("\n") for line in f]
        else:
            self.glob = glob.glob(g)
        self.branches = branches

    def dataframes(self, fs, maxfile=None, nproc=1, savemeta=False):
        if not isinstance(fs, list):
            fs = [fs]

        thisglob = self.glob 
        if maxfile:
            thisglob = thisglob[:maxfile]

        if nproc == "auto":
            nproc = min(CPU_COUNT, len(thisglob))

        ret = []

        try:
            with Pool(processes=nproc) as pool:
                for i, dfs in enumerate(tqdm(pool.imap_unordered(partial(_loaddf, fs), enumerate(thisglob)), total=len(thisglob), unit="file", delay=5, smoothing=0.2)):
                    if dfs is not None:
                        ret.append(dfs)
        # Ctrl-C handling
        except KeyboardInterrupt:
            print('Received Ctrl-C. Returning dataframes collected so far.')

        ret = [pd.concat([dfs[i] for dfs in ret], axis=0, ignore_index=False) for i in range(len(fs))] 

        return ret

        # Fix the index So that we don't need __ntuple
        for i in range(len(ret)):
            sub_index = ret[i].index.names[2:]
            ret[i] = ret[i].reset_index()
            ret[i].entry = ret[i].groupby(["__ntuple", "entry"]).ngroup()
            ret[i].set_index(["entry"] + sub_index, inplace=True, verify_integrity=True)
            ret[i].sort_index(inplace=True)
            if not savemeta:
                del ret[i]["__ntuple"]




In [7]:
def broadcast(v, df):
    for vi, ii in zip(v.index.names, df.index.names):
        if vi != ii:
            raise ValueError("Value index (%s) does not match index (%s)." % (str(vi), str(ii)))
    if len(v.index.names) > len(df.index.names):
        raise ValueError("Value index too long.")
    if len(v.index.names) == len(df.index.names):
        return v

    rpt = df.groupby(level=list(range(v.index.nlevels))).size()
    has_value = v.index.intersection(rpt.index)
    v_rpt = np.repeat(v.loc[has_value].values, rpt)

    return pd.Series(v_rpt, df.index).rename(v.name) 

def multicol_concat(lhs, rhs):
    # Fix the columns
    lhs_col = lhs.columns
    rhs_col = rhs.columns

    nlevel = max(lhs_col.nlevels, rhs_col.nlevels)

    def pad(c):
       return tuple(list(c) + [""]*(nlevel - len(c))) 

    lhs.columns = pd.MultiIndex.from_tuples([pad(c) for c in lhs_col])
    rhs.columns = pd.MultiIndex.from_tuples([pad(c) for c in rhs_col])

    return pd.concat([lhs, rhs], axis=1)

def multicol_add(df, s, **panda_kwargs):
    # if both the series and the df is one level, we can do a simple join()
    if isinstance(s.name, str) and df.columns.nlevels == 1:
        return df.join(s, **panda_kwargs)

    if isinstance(s.name, str):
        s.name = (s.name,)

    nlevel = max(df.columns.nlevels, len(s.name))
    def pad(c):
       return tuple(list(c) + [""]*(nlevel - len(c))) 

    if df.columns.nlevels < nlevel:
        df.columns = pd.MultiIndex.from_tuples([pad(c) for c in df.columns])
    if len(s.name) < nlevel:
        s.name = pad(s.name)

    return df.join(s, **panda_kwargs)

def multicol_merge(lhs, rhs, **panda_kwargs):
    # Fix the columns
    lhs_col = lhs.columns
    rhs_col = rhs.columns

    nlevel = max(lhs_col.nlevels, rhs_col.nlevels)

    def pad(c):
       nc = 1 if isinstance(c, str) else len(c)
       c0 = [c] if isinstance(c, str) else list(c)
       return tuple(c0 + [""]*(nlevel - nc)) 

    lhs.columns = pd.MultiIndex.from_tuples([pad(c) for c in lhs_col])
    rhs.columns = pd.MultiIndex.from_tuples([pad(c) for c in rhs_col])

    return lhs.merge(rhs, **panda_kwargs)

def detect_vectors(tree, branch):
    ret = []
    hierarchy = branch.split(".")
    for i in range(len(hierarchy)):
        subbranch = ".".join(hierarchy[:i+1])
        lenbranch = subbranch + "..length"
        if lenbranch in tree.keys():
            ret.append(subbranch)
    return ret

def idarray(ids, lens):
    return np.repeat(ids.values, lens.values)

def loadbranches(tree, branches, **uprargs):
    vectors = []
    for i,branch in enumerate(branches):
        this_vectors = detect_vectors(tree, branch)
        if i == 0:
            vectors = this_vectors
        elif len(this_vectors) == 0: # This case is ok since it will automatically broadcast
            pass
        # All the branches must have the same vector structure for this to work
        elif vectors != this_vectors:
            raise ValueError("Branches %s and %s have different vector structures in the CAF." % (branches[0], branch))

    lengths = [tree.arrays([v+"..length"], library="pd", **uprargs) for v in vectors]
    data = tree.arrays(branches, library="pd", **uprargs)

    # If there's no vectors, we can just return the top guy
    if len(lengths) == 0:
        data.index.name = "entry"
        df = data
    else:
        tomerge = lengths + [data]
        # Otherwise, iteratively merge the branches
        df = tomerge[0]
        df.index.name = "entry"

        # handle the rest
        for i in range(1, len(tomerge)):
            thismerge = tomerge[i]
            v_ind = i - 1

            # Build the information in the right-hand table needed to do the join
            # The "upidx" will be matched to the index vector-by-vector
            for i in range(v_ind):
                thismerge[vectors[v_ind] + "..upidx" + str(i)] = idarray(df[vectors[i]+ "..index"], df[vectors[v_ind] + "..length"])

            # Inner join! Throw away rows in the right-hand with no match in the left-hand
            df = pd.merge(df, thismerge, how="inner",
                         left_on = ["entry"] + [v+"..index" for v in vectors[:v_ind]],
                         right_on = ["entry"] + [vectors[v_ind] + "..upidx" + str(i) for i in range(v_ind)],
                         validate="one_to_many")

            # Make sure no rows in the right-hand were dropped
            assert(df.shape[0] == thismerge.shape[0])

            # postprocess: build the index
            df[vectors[v_ind] + "..index"] = df.groupby(["entry"] + [v+"..index" for v in vectors[:v_ind]]).cumcount()

        # Set the index
        df.set_index([v+"..index" for v in vectors], append=True, verify_integrity=True, inplace=True)

        # Drop all the metadata info we don't need anymore
        df = df[branches]

    # Setup branch names so df reflects structure of CAF file
    bsplit = [b.split(".") for b in branches]
    # Replace any reserved names
    def unreserve(s):
        if s == "index":
            return "idx"
        if s[0].isdigit(): # make the name a legal field 
            return "I" + s
        return s

    bsplit = [[unreserve(s) for s in b] for b in bsplit]

    depth = max([len(b) for b in bsplit])

    def pad(b):
        return tuple(b + [""]*(depth - len(b)))

    df.columns = pd.MultiIndex.from_tuples([pad(b) for b in bsplit])

    return df




In [8]:
mchdrbranches = [
    "rec.hdr.pot",
    "rec.hdr.first_in_subrun",
    "rec.hdr.ismc",
    "rec.hdr.run",
    "rec.hdr.subrun",
    "rec.hdr.ngenevt",
    "rec.hdr.evt",
    "rec.hdr.proc",
    "rec.hdr.cluster",
    "rec.hdr.fno",
]

hdrbranches = [
    "rec.hdr.pot",
    "rec.hdr.first_in_subrun",
    "rec.hdr.ismc",
    "rec.hdr.run",
    "rec.hdr.subrun",
    "rec.hdr.ngenevt",
    "rec.hdr.evt",
    "rec.hdr.proc",
    "rec.hdr.cluster",
    "rec.hdr.fno",

    # "rec.hdr.triggerinfo.trigger_id",
    # "rec.hdr.triggerinfo.gate_id",
    # "rec.hdr.triggerinfo.trigger_count",
    # "rec.hdr.triggerinfo.gate_count",
    # "rec.hdr.triggerinfo.gate_delta",
    # "rec.hdr.triggerinfo.global_trigger_time",
    # "rec.hdr.triggerinfo.prev_global_trigger_time",
]

potbranches = [
    "rec.hdr.numiinfo.spill_time_s",
    "rec.hdr.numiinfo.spill_time_ns",
    "rec.hdr.numiinfo.TRTGTD",
    "rec.hdr.numiinfo.TORTGT",
    "rec.hdr.numiinfo.daq_gates",
]

trueparticlenames = [
    "start_process",
    "end_process",
    "pdg",
    "startE",
    "start.x", "start.y", "start.z",
    "end.x", "end.y", "end.z",
    "genp.x", "genp.y", "genp.z",
    "length",
    "G4ID",
    "parent",
    "cont_tpc",
    "genE",
    "interaction_id"
]

trueparticlebranches = ["rec.true_particles.%s" % s for s in trueparticlenames]

pfpbranch = "rec.slc.reco.pfp."
trkbranch = pfpbranch + "trk."
shwbranch = pfpbranch + "shw."

pfobranches = [
    pfpbranch + "pfochar.chgendfrac",
    pfpbranch + "pfochar.chgfracspread",
    pfpbranch + "pfochar.linfitdiff",
    pfpbranch + "pfochar.linfitlen",
    pfpbranch + "pfochar.linfitgaplen",
    pfpbranch + "pfochar.linfitrms",
    pfpbranch + "pfochar.openanglediff",
    pfpbranch + "pfochar.pca2ratio",
    pfpbranch + "pfochar.pca3ratio", 
    pfpbranch + "pfochar.vtxdist" 
]

pfpbranches = [
    pfpbranch + "parent_is_primary",
    pfpbranch + "slcID",
    pfpbranch + "trackScore",
    pfpbranch + "parent",
    pfpbranch + "id",
    pfpbranch + "t0",
] + pfobranches

pfp_daughter_branch = [
    pfpbranch + "daughters"
]

trkbranches = [
    trkbranch + "producer",
    trkbranch + "start.x", trkbranch + "start.y", trkbranch + "start.z",
    trkbranch + "end.x", trkbranch + "end.y", trkbranch + "end.z",
    trkbranch + "dir.x", trkbranch + "dir.y", trkbranch + "dir.z",
    trkbranch + "len",
    trkbranch + "rangeP.p_muon",
    trkbranch + "mcsP.fwdP_muon",
    trkbranch + "rangeP.p_pion",
    trkbranch + "mcsP.fwdP_pion",
    trkbranch + "bestplane",
    trkbranch + "crthit.distance",
    trkbranch + "crthit.hit.time",
    trkbranch + "crthit.hit.pe",
    trkbranch + "chi2pid.2.pid_ndof",
    trkbranch + "chi2pid.2.chi2_muon",
    trkbranch + "chi2pid.2.chi2_proton",
    trkbranch + "chi2pid.2.pida",
] + pfpbranches

trkmcsbranches = [
  trkbranch + "mcsP.seg_length",
  trkbranch + "mcsP.seg_scatter_angles",
]

shwbranches = [
  shwbranch + "len"
]

trkhitadcbranches = [
  trkbranch + "calo.2.points.adcs"
]

trkhitbranches_perplane = lambda IPLANE : [
    trkbranch + "calo.%i.points.dedx"% IPLANE,
    trkbranch + "calo.%i.points.dqdx"% IPLANE,
    trkbranch + "calo.%i.points.pitch"% IPLANE,
    trkbranch + "calo.%i.points.integral"% IPLANE,
    trkbranch + "calo.%i.points.rr"% IPLANE,
    trkbranch + "calo.%i.points.wire"% IPLANE,
    trkbranch + "calo.%i.points.tpc"% IPLANE,
    trkbranch + "calo.%i.points.sumadc"% IPLANE,
    trkbranch + "calo.%i.points.t"% IPLANE,
    trkbranch + "calo.%i.points.x"% IPLANE,
    trkbranch + "calo.%i.points.y"% IPLANE,
    trkbranch + "calo.%i.points.z"% IPLANE,

    #trkbranch + "calo.%i.points.width"% IPLANE,
    #trkbranch + "calo.%i.points.mult"% IPLANE,
    #trkbranch + "calo.%i.points.tdc0"% IPLANE,

    trkbranch + "calo.%i.points.truth.h_e"% IPLANE,
    trkbranch + "calo.%i.points.truth.h_nelec"% IPLANE,
    trkbranch + "calo.%i.points.truth.pitch"% IPLANE,
    trkbranch + "calo.%i.points.truth.rr"% IPLANE,
]

trkhitbranches = trkhitbranches_perplane(2)
trkhitbranches_P1 = trkhitbranches_perplane(1)
trkhitbranches_P0 = trkhitbranches_perplane(0)

for n in trueparticlenames: trkbranches.append(trkbranch + "truth.p." + n)

slcbranches = [
    "rec.slc.is_clear_cosmic",
    "rec.slc.vertex.x", "rec.slc.vertex.y", "rec.slc.vertex.z",
    "rec.slc.self",
    "rec.slc.tmatch.eff",
    "rec.slc.tmatch.pur",
    "rec.slc.tmatch.index",
    "rec.slc.producer",
    "rec.slc.nuid.crlongtrkdiry"
]

mcbranches = [
    "rec.mc.nu.E",
    "rec.mc.nu.time",
    "rec.mc.nu.bjorkenX",
    "rec.mc.nu.inelasticityY",
    "rec.mc.nu.Q2",
    "rec.mc.nu.w",
    "rec.mc.nu.momentum.x",
    "rec.mc.nu.momentum.y",
    "rec.mc.nu.momentum.z",
    "rec.mc.nu.position.x",
    "rec.mc.nu.position.y",
    "rec.mc.nu.position.z",
    "rec.mc.nu.pdg",
    "rec.mc.nu.iscc",
    "rec.mc.nu.genie_mode",
    "rec.mc.nu.parent_pdg",
    "rec.mc.nu.parent_dcy_E",
]

mcprimbranches = [
    "rec.mc.nu.prim.genE",
    "rec.mc.nu.prim.length",
    "rec.mc.nu.prim.pdg",
    "rec.mc.nu.prim.genp.x",
    "rec.mc.nu.prim.genp.y",
    "rec.mc.nu.prim.genp.z",
    "rec.mc.nu.prim.start.x", "rec.mc.nu.prim.start.y", "rec.mc.nu.prim.start.z",
    "rec.mc.nu.prim.end.x", "rec.mc.nu.prim.end.y", "rec.mc.nu.prim.end.z",
]

slc_mcbranches = ["rec.slc.truth." + ".".join(s.split(".")[3:]) for s in mcbranches]
slc_mcprimbranches = ["rec.slc.truth." + ".".join(s.split(".")[3:]) for s in mcprimbranches]

mchbranches = [
  "rec.mc.prtl.time",
  "rec.mc.prtl.E",
  "rec.mc.prtl.M",
  "rec.mc.prtl.start.x", "rec.mc.prtl.start.y", "rec.mc.prtl.start.z",
  "rec.mc.prtl.enter.x", "rec.mc.prtl.enter.y", "rec.mc.prtl.enter.z",
  "rec.mc.prtl.exit.x", "rec.mc.prtl.exit.y", "rec.mc.prtl.exit.z",
  "rec.mc.prtl.decay_length",
  "rec.mc.prtl.allowed_decay_fraction",
  "rec.mc.prtl.C1",
  "rec.mc.prtl.C2",
  "rec.mc.prtl.C3",
  "rec.mc.prtl.C4",
  "rec.mc.prtl.C5",
]

stubbranches = [
    "rec.slc.reco.stub.vtx.x",
    "rec.slc.reco.stub.vtx.y",
    "rec.slc.reco.stub.vtx.z",
    "rec.slc.reco.stub.end.x",
    "rec.slc.reco.stub.end.y",
    "rec.slc.reco.stub.end.z",

    "rec.slc.reco.stub.efield_vtx",
    "rec.slc.reco.stub.efield_end",


    "rec.slc.reco.stub.truth.p.pdg",
    "rec.slc.reco.stub.truth.p.genE",
    "rec.slc.reco.stub.truth.p.interaction_id",
]

stubplanebranches = [
    "rec.slc.reco.stub.planes.p",
    "rec.slc.reco.stub.planes.hit_w",
    "rec.slc.reco.stub.planes.vtx_w",
    "rec.slc.reco.stub.planes.pitch",
    "rec.slc.reco.stub.planes.trkpitch",
]

stubhitbranches = [
    "rec.slc.reco.stub.planes.hits.charge",
    "rec.slc.reco.stub.planes.hits.ontrack",
    "rec.slc.reco.stub.planes.hits.wire",
]

eslc = "rec.dlp."

eslcbranches = [
    eslc + "is_neutrino",
    eslc + "nu_id",
    eslc + "num_particles",
    eslc + "num_primaries",
    eslc + "vertex.0",
    eslc + "vertex.1",
    eslc + "vertex.2",
]

eslcmatchedbranches = [
    eslc + "match",
]

eslcmatchovrlpbranches = [
    eslc + "match_overlap",
]

etruthint = "rec.dlp_true."

etruthintbranches = [
    etruthint + "id"
]

epart = "rec.dlp.particles."

eparticlebranches = [
    epart + "end_point.0",
    epart + "end_point.1",
    epart + "end_point.2",
    epart + "is_contained",
    epart + "is_primary",
    epart + "is_principal_match",
    epart + "is_valid",
    epart + "length",
    epart + "csda_ke",
    epart + "ke",
    epart + "momentum.0",
    epart + "momentum.1",
    epart + "momentum.2",
    epart + "pid",
    epart + "pid_scores.0",
    epart + "pid_scores.1",
    epart + "pid_scores.2",
    epart + "pid_scores.3",
    epart + "pid_scores.4",
    epart + "start_point.0",
    epart + "start_point.1",
    epart + "start_point.2",
    epart + "start_dir.0",
    epart + "start_dir.1",
    epart + "start_dir.2",
]

eparticlematchedbranches = [
    epart + "match",
]

eparticlematchovrlpbranches = [
    epart + "match_overlap",
]

etruthpart = "rec.dlp_true.particles."

etrueparticlebranches = [
    etruthpart + "track_id",
    etruthpart + "id",
]

etruthint = "rec.dlp_true."

etruthintbranches = [
    etruthint + "id",
    etruthint + "nu_id"
]




In [9]:
def make_hdrdf(f):
    hdr = loadbranches(f["recTree"], hdrbranches).rec.hdr
    return hdr

def make_mchdrdf(f):
    hdr = loadbranches(f["recTree"], mchdrbranches).rec.hdr
    return hdr

def make_potdf(f):
    pot = loadbranches(f["recTree"], potbranches).rec.hdr.numiinfo
    return pot

def make_mcnuwgtdf(f):
    return make_mcnudf(f, include_weights=True)

def make_mcnudf(f, include_weights=False):
    mcdf = make_mcdf(f)
    mcdf["ind"] = mcdf.index.get_level_values(1)
    if include_weights:
        wgtdf = pd.concat([numisyst.numisyst(mcdf.pdg, mcdf.E), geniesyst.geniesyst(f, mcdf.ind), g4syst.g4syst(f, mcdf.ind)], axis=1)
        mcdf = multicol_concat(mcdf, wgtdf)
    return mcdf

def make_mchdf(f, include_weights=False):
    mcdf = loadbranches(f["recTree"], mchbranches).rec.mc.prtl
    if include_weights:
        wgtdf = numisyst.numisyst(14, mcdf.E) # TODO: what PDG?
        mcdf = pd.concat([mcdf, wgtdf], axis=1)
    return mcdf

def make_trkdf(f, scoreCut=False, requiret0=False, requireCosmic=False, recalo=True, mcs=True):
    trkdf = loadbranches(f["recTree"], trkbranches + shwbranches)
    if scoreCut:
        trkdf = trkdf.rec.slc.reco[trkdf.rec.slc.reco.pfp.trackScore > 0.5]
    else:
        trkdf = trkdf.rec.slc.reco

    if requiret0:
        trkdf = trkdf[~np.isnan(trkdf.pfp.t0)]

    if requireCosmic:
        trkdf = trkdf[trkdf.pfp.parent == -1]

    if mcs:
        mcsdf = loadbranches(f["recTree"], [trkmcsbranches[0]]).rec.slc.reco.pfp.trk.mcsP
        mcsdf_angle = loadbranches(f["recTree"], [trkmcsbranches[1]]).rec.slc.reco.pfp.trk.mcsP
        mcsdf_angle.index.set_names(mcsdf.index.names, inplace=True)

        mcsdf = mcsdf.merge(mcsdf_angle, how="left", left_index=True, right_index=True)
        mcsgroup = list(range(mcsdf.index.nlevels-1))
        cumlen = mcsdf.seg_length.groupby(level=mcsgroup).cumsum()*14 # convert rad length to cm
        maxlen = (cumlen*(mcsdf.seg_scatter_angles >= 0)).groupby(level=mcsgroup).max()
        trkdf[("pfp", "trk", "mcsP", "len", "", "")] = maxlen


    trkdf[("pfp", "tindex", "", "", "", "")] = trkdf.index.get_level_values(2)

    return trkdf

def make_trkhitdf(f):
    df = loadbranches(f["recTree"], trkhitbranches).rec.slc.reco.pfp.trk.calo.I2.points

    # Firsthit and Lasthit info
    ihit = df.index.get_level_values(-1)
    df["firsthit"] = ihit == 0

    lasthit = df.groupby(level=list(range(df.index.nlevels-1))).tail(1).copy()
    lasthit["lasthit"] = True
    df["lasthit"] = lasthit.lasthit
    df.lasthit = df.lasthit.fillna(False)

    return df

def make_slcdf(f):
    slcdf = loadbranches(f["recTree"], slcbranches)
    slcdf = slcdf.rec

    slc_mcdf = make_mcdf(f, slc_mcbranches, slc_mcprimbranches)
    slc_mcdf.columns = pd.MultiIndex.from_tuples([tuple(["slc", "truth"] + list(c)) for c in slc_mcdf.columns])
    slcdf = multicol_merge(slcdf, slc_mcdf, left_index=True, right_index=True, how="left", validate="one_to_one")

    return slcdf

def make_mcdf(f, branches=mcbranches, primbranches=mcprimbranches):
    # load the df
    mcdf = loadbranches(f["recTree"], branches)
    while mcdf.columns.nlevels > 2:
        mcdf.columns = mcdf.columns.droplevel(0)

    # Add in primary particle info
    mcprimdf = loadbranches(f["recTree"], primbranches)
    while mcprimdf.columns.nlevels > 2:
        mcprimdf.columns = mcprimdf.columns.droplevel(0)

    mcprimdf.index = mcprimdf.index.rename(mcdf.index.names[:2] + mcprimdf.index.names[2:])

    PROTON_MASS = 0.938272
    max_proton_KE = mcprimdf[np.abs(mcprimdf.pdg)==2212].genE.groupby(level=[0,1]).max() - PROTON_MASS
    max_proton_KE.name = ("max_proton_ke", "")
    mcdf = mcdf.join(max_proton_KE)

    mcdf.max_proton_ke = mcdf.max_proton_ke.fillna(0.)

    # particle counts
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==2112).groupby(level=[0,1]).sum().rename(("nn", "")))
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==2212).groupby(level=[0,1]).sum().rename(("np", "")))
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==13).groupby(level=[0,1]).sum().rename(("nmu", "")))
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==211).groupby(level=[0,1]).sum().rename(("npi", "")))
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==111).groupby(level=[0,1]).sum().rename(("npi0", "")))
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==22).groupby(level=[0,1]).sum().rename(("ng", "")))
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==321).groupby(level=[0,1]).sum().rename(("nk", "")))
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==310).groupby(level=[0,1]).sum().rename(("nk0", "")))
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==3112).groupby(level=[0,1]).sum().rename(("nsm", "")))
    mcdf = mcdf.join((np.abs(mcprimdf.pdg)==3222).groupby(level=[0,1]).sum().rename(("nsp", "")))

    # particle counts w/ threshold
    proton_KE = mcprimdf[np.abs(mcprimdf.pdg)==2212].genE - PROTON_MASS
    mcdf = mcdf.join(((np.abs(mcprimdf.pdg)==2212) & (proton_KE > 0.05)).groupby(level=[0,1]).sum().rename(("np_50MeV","")))
    mcdf = mcdf.join(((np.abs(mcprimdf.pdg)==2212) & (proton_KE > 0.02)).groupby(level=[0,1]).sum().rename(("np_20MeV","")))
 
    # lepton info
    mudf = mcprimdf[np.abs(mcprimdf.pdg)==13].sort_values(mcprimdf.index.names[:2] + [("genE", "")]).groupby(level=[0,1]).last()
    mudf.columns = pd.MultiIndex.from_tuples([tuple(["mu"] + list(c)) for c in mudf.columns])

    pdf = mcprimdf[mcprimdf.pdg==2212].sort_values(mcprimdf.index.names[:2] + [("genE", "")]).groupby(level=[0,1]).last()
    pdf.columns = pd.MultiIndex.from_tuples([tuple(["p"] + list(c)) for c in pdf.columns])

    mcdf = multicol_merge(mcdf, mudf, left_index=True, right_index=True, how="left", validate="one_to_one")
    mcdf = multicol_merge(mcdf, pdf, left_index=True, right_index=True, how="left", validate="one_to_one")

    return mcdf

def make_slc_trkdf(f, trkScoreCut=False, trkDistCut=10., cutClearCosmic=True, **trkArgs):
    # load
    trkdf = make_trkdf(f, trkScoreCut, **trkArgs)
    slcdf = make_slcdf(f)

    # merge in tracks
    slcdf = multicol_merge(slcdf, trkdf, left_index=True, right_index=True, how="right", validate="one_to_many")

    # distance from vertex to track start
    slcdf = multicol_add(slcdf, dmagdf(slcdf.slc.vertex, slcdf.pfp.trk.start).rename(("pfp", "dist_to_vertex")))

    if trkDistCut > 0:
        slcdf = slcdf[slcdf.pfp.dist_to_vertex < trkDistCut]
    if cutClearCosmic:
        slcdf = slcdf[slcdf.slc.is_clear_cosmic==0]

    return slcdf

def make_eslc_partdf(f, trkDistCut=-1, **trkArgs):
    # load
    partdf = make_epartdf(f, **trkArgs)
    partdf.columns = pd.MultiIndex.from_tuples([tuple(["particle"] + list(c)) for c in partdf.columns])
    eslcdf = make_eslcdf(f)

    # merge in tracks
    eslcdf = multicol_merge(eslcdf, partdf, left_index=True, right_index=True, how="right", validate="one_to_many")
    eslcdf = multicol_add(eslcdf, dmagdf(eslcdf.vertex, eslcdf.particle.start_point).rename("dist_to_vertex"))

    if trkDistCut > 0:
        eslcdf = eslcdf[eslcdf.dist_to_vertex < trkDistCut]

    return eslcdf

alpha = 0.930                                                                                                                                             
LAr_density_gmL = 1.38434                                                                                                                                             
Efield = 0.5                                                                                                                                              
beta = 0.212 / (LAr_density_gmL * Efield)                                                                                                                             
Wion = 1e3 / 4.237e7                                                                                                                                      

def make_stubs(f):
    stubdf = loadbranches(f["recTree"], stubbranches)
    stubdf = stubdf.rec.slc.reco.stub

    stubpdf = loadbranches(f["recTree"], stubplanebranches)
    stubpdf = stubpdf.rec.slc.reco.stub.planes

    stubdf["nplane"] = stubpdf.groupby(level=[0,1,2]).size()
    stubdf["plane"] = stubpdf.p.groupby(level=[0,1,2]).first()

    stubhitdf = loadbranches(f["recTree"], stubhitbranches)
    stubhitdf = stubhitdf.rec.slc.reco.stub.planes.hits

    stubhitdf = stubhitdf.join(stubpdf)
    stubhitdf = stubhitdf.join(stubdf.efield_vtx)
    stubhitdf = stubhitdf.join(stubdf.efield_end)

    hdrdf = make_mchdrdf(f)
    ismc = hdrdf.ismc.iloc[0]
    def dEdx2dQdx_mc(dEdx): # MC parameters
        # beta = MODB_mc / (LAr_density_gmL_mc * Efield_mc)
        # alpha = MODA_mc
        return np.log(alpha + dEdx*beta) / (Wion*beta)
    def dEdx2dQdx_data(dEdx): # data parameters
        # beta = MODB_data / (LAr_density_gmL_data * Efield_data)
        # alpha = MODA_data
        return np.log(alpha + dEdx*beta) / (Wion*beta)

    dEdx2dQdx = dEdx2dQdx_mc if ismc else dEdx2dQdx_data
    MIP_dqdx = dEdx2dQdx(1.7) 

    stub_end_charge = stubhitdf.charge[stubhitdf.wire == stubhitdf.hit_w].groupby(level=[0,1,2,3]).first().groupby(level=[0,1,2]).first()
    stub_end_charge.name = ("endp_charge", "", "")

    stub_pitch = stubpdf.pitch.groupby(level=[0,1,2]).first()
    stub_pitch.name = ("pitch", "", "")

    stubdir_is_pos = (stubhitdf.hit_w - stubhitdf.vtx_w) > 0.
    when_sum = ((stubhitdf.wire > stubhitdf.vtx_w) == stubdir_is_pos) & (((stubhitdf.wire < stubhitdf.hit_w) == stubdir_is_pos) | (stubhitdf.wire == stubhitdf.hit_w)) 
    stubcharge = (stubhitdf.charge[when_sum]).groupby(level=[0,1,2,3]).sum().groupby(level=[0,1,2]).first()
    stubcharge.name = ("charge", "", "")

    stubinccharge = (stubhitdf.charge).groupby(level=[0,1,2,3]).sum().groupby(level=[0,1,2]).first()
    stubinccharge.name = ("inc_charge", "", "")

    hit_before_start = ((stubhitdf.wire < stubhitdf.vtx_w) == stubdir_is_pos)
    stub_inc_sub_charge = (stubhitdf.charge - MIP_dqdx*stubhitdf.ontrack*(~hit_before_start)*stubhitdf.trkpitch).groupby(level=[0,1,2,3]).sum().groupby(level=[0,1,2]).first()
    stub_inc_sub_charge.name = ("inc_sub_charge", "", "")

    stubdf = stubdf.join(stubcharge)
    stubdf = stubdf.join(stubinccharge)
    stubdf = stubdf.join(stub_inc_sub_charge)
    stubdf = stubdf.join(stub_end_charge)
    stubdf = stubdf.join(stub_pitch)
    stubdf["length"] = magdf(stubdf.vtx - stubdf.end)
    stubdf["Q"] = stubdf.inc_sub_charge

    # convert charge to energy
    if ismc:
        stubdf["ke"] = Q2KE_mc(stubdf.Q)
        # also do calorimetric variations
        stubdf["ke_callo"] = Q2KE_mc_callo(stubdf.Q)
        stubdf["ke_calhi"] = Q2KE_mc_calhi(stubdf.Q)
    else:
        stubdf["ke"] = Q2KE_data(stubdf.Q)
        stubdf["ke_callo"] = np.nan
        stubdf["ke_calhi"] = np.nan

    stubdf.ke = stubdf.ke.fillna(0)
    stubdf.Q = stubdf.Q.fillna(0)

    stubdf["dedx"] = stubdf.ke / stubdf.length
    stubdf["dedx_callo"] = stubdf.ke_callo / stubdf.length
    stubdf["dedx_calhi"] = stubdf.ke_calhi / stubdf.length

    # only take collection plane
    stubdf = stubdf[stubdf.plane == 2]

    stub_length_bins = [0, 0.5, 1, 2, 3]
    stub_length_name = ["l0_5cm", "l1cm", "l2cm", "l3cm"]
    tosave = ["dedx", "dedx_callo", "dedx_calhi", "Q", "length", "charge", "inc_charge"] 

    df_tosave = []
    for blo, bhi, name in zip(stub_length_bins[:-1], stub_length_bins[1:], stub_length_name):
        stub_tosave = stubdf.dedx[(stubdf.length > blo) & (stubdf.length < bhi)].groupby(level=[0,1]).idxmax()
        for col in tosave:
            s = stubdf.loc[stub_tosave, col]
            s.name = ("stub", name, col, "", "", "")
            s.index = s.index.droplevel(-1)
            df_tosave.append(s)

    return pd.concat(df_tosave, axis=1)

def make_eslcdf(f):
    eslcdf = loadbranches(f["recTree"], eslcbranches)
    eslcdf = eslcdf.rec.dlp

    etintdf = loadbranches(f["recTree"], etruthintbranches)
    etintdf = etintdf.rec.dlp_true
    
    # match to the truth info
    mcdf = make_mcdf(f)
    # mc is truth
    mcdf.columns = pd.MultiIndex.from_tuples([tuple(["truth"] + list(c)) for c in mcdf.columns])

    # Do matching
    # 
    # First get the ML true particle IDs matched to each reco particle
    eslc_matchdf = loadbranches(f["recTree"], eslcmatchedbranches)
    eslc_match_overlap_df = loadbranches(f["recTree"], eslcmatchovrlpbranches)
    eslc_match_overlap_df.index.names = eslc_matchdf.index.names

    eslc_matchdf = multicol_merge(eslc_matchdf, eslc_match_overlap_df, left_index=True, right_index=True, how="left", validate="one_to_one")
    eslc_matchdf = eslc_matchdf.rec.dlp

    # Then use bestmatch.match to get the nu ids in etintdf
    eslc_matchdf_wids = pd.merge(eslc_matchdf, etintdf, left_on=["entry", "match"], right_on=["entry", "id"], how="left")
    eslc_matchdf_wids.index = eslc_matchdf.index

    # Now use nu_ids to get the true interaction information
    eslc_matchdf_trueints = multicol_merge(eslc_matchdf_wids, mcdf, left_on=["entry", "nu_id"], right_index=True, how="left")
    eslc_matchdf_trueints.index = eslc_matchdf_wids.index

    # delete unnecesary matching branches
    del eslc_matchdf_trueints[("match", "")]
    del eslc_matchdf_trueints[("nu_id", "")]
    del eslc_matchdf_trueints[("id", "")]

    # first match is best match
    bestmatch = eslc_matchdf_trueints.groupby(level=list(range(eslc_matchdf_trueints.index.nlevels-1))).first()

    # add extra levels to eslcdf columns
    eslcdf.columns = pd.MultiIndex.from_tuples([tuple(list(c) + [""]*2) for c in eslcdf.columns])

    eslcdf_withmc = multicol_merge(eslcdf, bestmatch, left_index=True, right_index=True, how="left")

    # Fix position names (I0, I1, I2) -> (x, y, z)
    def mappos(s):
        if s == "I0": return "x"
        if s == "I1": return "y"
        if s == "I2": return "z"
        return s
    def fixpos(c):
        if c[0] not in ["end_point", "start_point", "start_dir", "vertex", "momentum"]: return c
        return tuple([c[0]] + [mappos(c[1])] + list(c[2:]))

    eslcdf_withmc.columns = pd.MultiIndex.from_tuples([fixpos(c) for c in eslcdf_withmc.columns])

    return eslcdf_withmc

def make_epartdf(f):
    epartdf = loadbranches(f["recTree"], eparticlebranches)
    epartdf = epartdf.rec.dlp.particles

    tpartdf = loadbranches(f["recTree"], trueparticlebranches)
    tpartdf = tpartdf.rec.true_particles
    # cut out EMShowerDaughters
    # tpartdf = tpartdf[(tpartdf.parent == 0)]

    etpartdf = loadbranches(f["recTree"], etrueparticlebranches)
    etpartdf = etpartdf.rec.dlp_true.particles
    etpartdf.columns = [s for s in etpartdf.columns]
    
    # Do matching
    # 
    # First get the ML true particle IDs matched to each reco particle
    epart_matchdf = loadbranches(f["recTree"], eparticlematchedbranches)
    epart_match_overlap_df = loadbranches(f["recTree"], eparticlematchovrlpbranches)
    epart_match_overlap_df.index.names = epart_matchdf.index.names
    epart_matchdf = multicol_merge(epart_matchdf, epart_match_overlap_df, left_index=True, right_index=True, how="left", validate="one_to_one")
    epart_matchdf = epart_matchdf.rec.dlp.particles
    # get the best match (highest match_overlap), assume it's sorted
    bestmatch = epart_matchdf.groupby(level=list(range(epart_matchdf.index.nlevels-1))).first()
    bestmatch.columns = [s for s in bestmatch.columns]

    # Then use betmatch.match to get the G4 track IDs in etpartdf
    bestmatch_wids = pd.merge(bestmatch, etpartdf, left_on=["entry", "match"], right_on=["entry", "id"], how="left")
    bestmatch_wids.index = bestmatch.index

    # Now use the G4 track IDs to get the true particle information
    bestmatch_trueparticles = multicol_merge(bestmatch_wids, tpartdf, left_on=["entry", "track_id"], right_on=["entry", ("G4ID", "")], how="left")
    bestmatch_trueparticles.index = bestmatch_wids.index

    # delete unnecesary matching branches
    del bestmatch_trueparticles[("match", "")]
    del bestmatch_trueparticles[("track_id", "")]
    del bestmatch_trueparticles[("id", "")]

    # add extra level to epartdf columns
    epartdf.columns = pd.MultiIndex.from_tuples([tuple(list(c) + [""]) for c in epartdf.columns])

    # put everything in epartdf
    for c in bestmatch_trueparticles.columns:
        epartdf[tuple(["truth"] + list(c))] = bestmatch_trueparticles[c]

    # Fix position names (I0, I1, I2) -> (x, y, z)
    def mappos(s):
        if s == "I0": return "x"
        if s == "I1": return "y"
        if s == "I2": return "z"
        return s
    def fixpos(c):
        if c[0] not in ["end_point", "start_point", "start_dir", "vertex"]: return c
        return tuple([c[0]] + [mappos(c[1])] + list(c[2:]))

    epartdf.columns = pd.MultiIndex.from_tuples([fixpos(c) for c in epartdf.columns])

    return epartdf

def make_trkdf(f, scoreCut=False, requiret0=False, requireCosmic=False, recalo=True, mcs=True):
    trkdf = loadbranches(f["recTree"], trkbranches + shwbranches)
    if scoreCut:
        trkdf = trkdf.rec.slc.reco[trkdf.rec.slc.reco.pfp.trackScore > 0.5]
    else:
        trkdf = trkdf.rec.slc.reco

    if requiret0:
        trkdf = trkdf[~np.isnan(trkdf.pfp.t0)]

    if requireCosmic:
        trkdf = trkdf[trkdf.pfp.parent == -1]

    if mcs:
        mcsdf = loadbranches(f["recTree"], [trkmcsbranches[0]]).rec.slc.reco.pfp.trk.mcsP
        mcsdf_angle = loadbranches(f["recTree"], [trkmcsbranches[1]]).rec.slc.reco.pfp.trk.mcsP
        mcsdf_angle.index.set_names(mcsdf.index.names, inplace=True)

        mcsdf = mcsdf.merge(mcsdf_angle, how="left", left_index=True, right_index=True)
        mcsgroup = list(range(mcsdf.index.nlevels-1))
        cumlen = mcsdf.seg_length.groupby(level=mcsgroup).cumsum()*14 # convert rad length to cm
        maxlen = (cumlen*(mcsdf.seg_scatter_angles >= 0)).groupby(level=mcsgroup).max()
        trkdf[("pfp", "trk", "mcsP", "len", "", "")] = maxlen


    trkdf[("pfp", "tindex", "", "", "", "")] = trkdf.index.get_level_values(2)

    return trkdf

def make_eevtdf(f):
    # load slices and particles
    partdf = make_epartdf(f)

    df = make_eslcdf(f)

    # load the proton and muon candidates
    primary = partdf.is_primary
    mudf = partdf[primary & (partdf.pid == 2)].sort_values(partdf.index.names[:2] + [("length", "", "")]).groupby(level=[0,1]).last()
    mudf.columns = pd.MultiIndex.from_tuples([tuple(["mu"] + list(c)) for c in mudf.columns])

    pdf = partdf[primary & (partdf.pid == 4)].sort_values(partdf.index.names[:2] + [("length", "", "")]).groupby(level=[0,1]).last()
    pdf.columns = pd.MultiIndex.from_tuples([tuple(["p"] + list(c)) for c in pdf.columns])

    df = multicol_merge(df, mudf, left_index=True, right_index=True, how="left", validate="one_to_one")
    df = multicol_merge(df, pdf, left_index=True, right_index=True, how="left", validate="one_to_one")

    # in case we want to cut out other objects -- save the highest energy of each other particle
    lead_gamma_energy = partdf.ke[primary & (partdf.pid == 0)].groupby(level=[0,1]).max().rename("lead_gamma_energy")
    df = multicol_add(df, lead_gamma_energy)

    lead_elec_energy = partdf.ke[primary & (partdf.pid == 1)].groupby(level=[0,1]).max().rename("lead_elec_energy")
    df = multicol_add(df, lead_elec_energy)

    lead_pion_length = partdf.length[primary & (partdf.pid == 3)].groupby(level=[0,1]).max().rename("lead_pion_length")
    df = multicol_add(df, lead_pion_length)

    subl_muon_length = partdf[primary & (partdf.pid == 2)].sort_values(partdf.index.names[:2] + [("length", "", "")]).length.groupby(level=[0,1]).nth(-2).rename("subl_muon_length")
    df = multicol_add(df, subl_muon_length)

    subl_proton_length = partdf[primary & (partdf.pid == 4)].sort_values(partdf.index.names[:2] + [("length", "", "")]).length.groupby(level=[0,1]).nth(-2).rename("subl_proton_length")
    df = multicol_add(df, subl_proton_length)

    # Apply pre-selection: Require fiducial vertex, at least one muon, at least one proton

    # require both muon and proton to be present
    df = df[~np.isnan(df.mu.pid) & ~np.isnan(df.p.pid)]

    # require fiducial verex
    df = df[InFV(df.vertex, 50)]

    return df


In [46]:
import sys
import os
# from pyanalib.ntuple_glob import NTupleGlob
# from makedf.makedf import *
import pandas as pd
import warnings
from tables import NaturalNameWarning

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=NaturalNameWarning)

inputs = [fname]
inputs =  "/pnfs/sbnd/scratch/users/lynnt/miniprod_rockbox/*/output.flat.root"
ntuples = NTupleGlob(inputs, None)

In [62]:
f = uproot.open(fname, timeout=120)

In [49]:
DFS = [make_hdrdf, make_mcnudf, make_trkdf, make_slcdf, make_mcdf, make_slc_trkdf]
NAMES = ["hdr", "mcnu", "trk", "slc", "mc", "slc_trk"]
ntuples = NTupleGlob(inputs, None)
dfs = ntuples.dataframes(nproc="auto", fs=DFS)

with pd.HDFStore("/exp/sbnd/data/users/munjung/sbnd_gump.df") as hdf:
    for k,df in zip(reversed(NAMES), reversed(dfs)): # go in reverse order so we can delete along the way
        try:
            hdf.put(key=k, value=df, format="fixed")
        except Exception as e:
            print("Table %s failed to save, skipping. Exception: %s" % (k, str(e)))

  0%|          | 1/1643 [01:40<45:54:58, 100.67s/file]

Error processing file (/pnfs/sbnd/scratch/users/lynnt/miniprod_rockbox/11293023_952/output.flat.root). Skipping...
expected Chunk of length 253776,
received 0 bytes from MemmapSource
for file path /pnfs/sbnd/scratch/users/lynnt/miniprod_rockbox/11293023_952/output.flat.root


In [42]:
# make_hdrdf(f)
# make_mcnudf(f)
# make_trkdf(f)
# make_slcdf(f)
# make_mcdf(f)
# make_slc_trkdf(f)

slc              \
                                             is_clear_cosmic      vertex   
                                                                       x   
                                                                           
                                                                           
                                                                           
entry rec.slc..index rec.slc.reco.pfp..index                               
0     0              0                                     0  -56.326027   
      1              0                                     0  107.954773   
1     0              0                                     0   25.229219   
                     1                                     0   25.229219   
2     0              0                                     0 -148.088028   
..                                                       ...         ...   
171   0              1                                     0  187.436020   
                     2                                     0  187.436020   
      1              0                                     0 -150.344925   
172   0              0                                     0   39.699600   
                     1                                     0   39.699600   

                                                                           \
                                                                     self   
                                                       y           z        
                                                                            
                                                                            
                                                                            
entry rec.slc..index rec.slc.reco.pfp..index                                
0     0              0                         14.985526    3.066065   17   
      1              0                        200.184189  484.257202   18   
1     0              0                        184.784729  430.343231   29   
                     1                        184.784729  430.343231   29   
2     0              0                       -136.450851  390.385559   31   
..                                                   ...         ...  ...   
171   0              1                         82.953987  126.170845   62   
                     2                         82.953987  126.170845   62   
      1              0                        200.057495   44.213875   63   
172   0              0                        177.848648  127.500061   91   
                     1                        177.848648  127.500061   91   

                                                                       \
                                                tmatch                  
                                                   eff       pur  idx   
                                                                        
                                                                        
                                                                        
entry rec.slc..index rec.slc.reco.pfp..index                            
0     0              0                        0.985187  0.658556    0   
      1              0                             NaN       NaN -999   
1     0              0                        0.899220  0.783142    0   
                     1                        0.899220  0.783142    0   
2     0              0                             NaN       NaN -999   
..                                                 ...       ...  ...   
171   0              1                        0.980937  0.979137    1   
                     2                        0.980937  0.979137    1   
      1              0                             NaN       NaN -999   
172   0              0                             NaN       NaN -999   
                     1                             NaN      

In [21]:
eslcdf = make_eslcdf(f) #.truth

In [22]:
slcdf = make_slcdf(f) #.slc.truth.E

In [23]:
eslcdf

is_neutrino nu_id num_particles num_primaries  \
                                                                     
                                                                     
                                                                     
entry rec.dlp..index                                                 
0     0                        0    -1             4             2   
      1                        0    -1             2             1   
      2                        0    -1             8             1   
      3                        0    -1             4             1   
      4                        0    -1             3             1   
...                          ...   ...           ...           ...   
10    29                       0    -1             4             1   
      30                       0    -1             1             1   
      31                       0    -1             1             1   
      32                       0    -1             4             1   
      33                       0    -1             3             1   

                          vertex                         match_overlap truth  \
                               x           y           z                   E   
                                                                               
                                                                               
entry rec.dlp..index                                                           
0     0               -63.840000  134.789993  853.898987      0.966405   NaN   
      1               -99.239998   83.190002 -856.101013      0.334062   NaN   
      2              -161.639999 -180.509995 -141.501007      0.898836   NaN   
      3              -250.740005 -158.910004  -53.901001      0.906977   NaN   
      4              -265.140015 -131.610001  567.999023      0.955863   NaN   
...                          ...         ...         ...           ...   ...   
10    29              141.660004  133.289993  102.399002      0.624894   NaN   
      30              156.059998   -8.610000  285.998993      0.472865   NaN   
      31              155.460007 -128.610001  314.199005      1.000000   NaN   
      32              264.660004   35.490002  366.699005      0.505854   NaN   
      33               67.559998   11.790000  416.498993      0.601986   NaN   

                           ...                                             
                     time  ...   p                                         
                           ... pdg genp         start         end          
                           ...        x   y   z     x   y   z   x   y   z  
entry rec.dlp..index       ...                                             
0     0               NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  
      1               NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  
      2               NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  
      3               NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  
      4               NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  
...                   ...  ...  ..  ...  ..  ..   ...  ..  ..  ..  ..  ..  
10    29              NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  
      30              NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  
      31              NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  
      32              NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  
      33              NaN  ... NaN  NaN NaN NaN   NaN NaN NaN NaN NaN NaN  

[419 rows x 62 columns]

In [24]:
slcdf

slc                                           \
                     is_clear_cosmic      vertex                         self   
                                               x           y           z        
                                                                                
                                                                                
entry rec.slc..index                                                            
0     0                            0 -318.476349 -141.180664  894.770874  187   
      1                            1 -182.839462  133.310226 -676.061890    0   
      2                            1  -63.651737  135.380020  854.096069    1   
      3                            1 -199.596786  134.244171 -156.474030    2   
      4                            1 -127.817108  134.903488 -275.982941    3   
...                              ...         ...         ...         ...  ...   
10    23                           1  166.934998   77.669617  444.225647    6   
      24                           1   21.831558  134.801483 -273.252197    7   
      25                           1    7.997945  -56.509514  127.820389    8   
      26                           1  155.254990 -109.299454 -509.747650   10   
      27                           1  155.421722 -128.727982  314.250885   16   

                                                             ...             \
                     tmatch          producer          nuid  ... truth        
                        eff pur  idx          crlongtrkdiry  ...     p        
                                                             ...   pdg genp   
                                                             ...          x   
entry rec.slc..index                                         ...              
0     0                 NaN NaN -999        0     -0.812481  ...   NaN  NaN   
      1                 NaN NaN -999        0  -9999.000000  ...   NaN  NaN   
      2                 NaN NaN -999        0  -9999.000000  ...   NaN  NaN   
      3                 NaN NaN -999        0  -9999.000000  ...   NaN  NaN   
      4                 NaN NaN -999        0  -9999.000000  ...   NaN  NaN   
...                     ...  ..  ...      ...           ...  ...   ...  ...   
10    23                NaN NaN -999        1  -9999.000000  ...   NaN  NaN   
      24                NaN NaN -999        1  -9999.000000  ...   NaN  NaN   
      25                NaN NaN -999        1  -9999.000000  ...   NaN  NaN   
      26                NaN NaN -999        1  -9999.000000  ...   NaN  NaN   
      27                NaN NaN -999        1  -9999.000000  ...   NaN  NaN   

                                                        
                                                        
                                                        
                             start         end          
                       y   z     x   y   z   x   y   z  
entry rec.slc..index                                    
0     0              NaN NaN   NaN NaN NaN NaN NaN NaN  
      1              NaN NaN   NaN NaN NaN NaN NaN NaN  
      2              NaN NaN   NaN NaN NaN NaN NaN NaN  
      3              NaN NaN   NaN NaN NaN NaN NaN NaN  
      4              NaN NaN   NaN NaN NaN NaN NaN NaN  
...                   ..  ..   ...  ..  ..  ..  ..  ..  
10    23             NaN NaN   NaN NaN NaN NaN NaN NaN  
      24             NaN NaN   NaN NaN NaN NaN NaN NaN  
      25             NaN NaN   NaN NaN NaN NaN NaN NaN  
      26             NaN NaN   NaN NaN NaN NaN NaN NaN  
      27             NaN NaN   NaN NaN NaN NaN NaN NaN  

[332 rows x 64 columns]

In [65]:
def make_evtdf_sbnd(f):

    mcdf = make_mcdf(f)
    slcdf = make_slc_trkdf(f)

    # PID
    ts_cut = (slcdf.pfp.trackScore > 0.5)

    pid_shw = np.invert(ts_cut)

    # muon
    MUSEL_MUSCORE_TH = 25
    MUSEL_PSCORE_TH = 100
    MUSEL_LEN_TH = 50

    # TODO: used BDT scores
    # muon_chi2 = (Avg(df, "muon", drop_0=True) < MUSEL_MUSCORE_TH) & (Avg(df, "proton", drop_0=True) > MUSEL_PSCORE_TH)
    # len_cut = (masterdf.len.squeeze() > MUSEL_LEN_TH)
    # dazzle_muon = (masterdf.dazzle.muonScore > 0.6)
    # muon_cut = (muon_chi2) & (len_cut | dazzle_muon)

    mu_score_cut = (slcdf.pfp.trk.chi2pid.I2.chi2_muon < MUSEL_MUSCORE_TH) & \
        (slcdf.pfp.trk.chi2pid.I2.chi2_proton > MUSEL_PSCORE_TH)
    mu_len_cut = (slcdf.pfp.trk.len > MUSEL_LEN_TH)
    mu_cut = (mu_score_cut) & (mu_len_cut)
    pid_mu = (ts_cut) & (mu_cut)

    # proton 
    PSEL_MUSCORE_TH = 0
    PSEL_PSCORE_TH = 90
    p_score_cut = (slcdf.pfp.trk.chi2pid.I2.chi2_muon > PSEL_MUSCORE_TH) & (slcdf.pfp.trk.chi2pid.I2.chi2_muon < PSEL_PSCORE_TH) 
    p_cut = np.invert(mu_cut) & p_score_cut
    pid_p = (ts_cut) & (p_cut)

    # rest is pion
    pi_cut = np.invert(mu_cut | p_cut)
    pid_pi = (ts_cut) & (pi_cut)

    # TODO: don't use trackscore

    # ---------------------------

    # store PID info
    slcdf[("pfp", "pid", "", "", "", "")] = np.nan
    slcdf.loc[pid_shw, ("pfp","pid")] = -1
    slcdf.loc[pid_mu, ("pfp","pid")] = 13
    slcdf.loc[pid_p, ("pfp","pid")] = 2212
    slcdf.loc[pid_pi, ("pfp","pid")] = 211

    mudf = slcdf[(slcdf.pfp.pid == 13)].sort_values(slcdf.pfp.index.names[:-1] + [("pfp", "trk", "len", "", "", "")]).groupby(level=[0,1,2]).last()
    mudf.columns = pd.MultiIndex.from_tuples([tuple(["mu"] + list(c)) for c in mudf.columns])

    pdf = slcdf[(slcdf.pfp.pid == 2212)].sort_values(slcdf.pfp.index.names[:-1] + [("pfp", "trk", "len", "", "", "")]).groupby(level=[0,1,2]).last()
    pdf.columns = pd.MultiIndex.from_tuples([tuple(["p"] + list(c)) for c in pdf.columns])

    slcdf = multicol_merge(slcdf, mudf, left_index=True, right_index=True, how="left", validate="one_to_one")
    slcdf = multicol_merge(slcdf, pdf, left_index=True, right_index=True, how="left", validate="one_to_one")

    # in case we want to cut out other objects -- save the highest energy of each other particle
    lead_shw_length = slcdf.pfp.trk.len[(slcdf.pfp.pid < 0)].groupby(level=[0,1,2]).max().rename("lead_shw_length")
    slcdf = multicol_add(slcdf, lead_shw_length)

    lead_pion_length = slcdf.pfp.trk.len[(slcdf.pfp.pid == 211)].groupby(level=[0,1,2]).max().rename("lead_pion_length")
    slcdf = multicol_add(slcdf, lead_pion_length)

    subl_muon_length = slcdf[(slcdf.pfp.pid == 13)].sort_values(slcdf.pfp.index.names[:-1] + [("pfp", "trk", "len", "", "", "")]).pfp.trk.len.groupby(level=[0,1,2]).nth(-2).rename("subl_muon_length")
    slcdf = multicol_add(slcdf, subl_muon_length)

    subl_proton_length = slcdf[(slcdf.pfp.pid == 2212)].sort_values(slcdf.pfp.index.names[:-1] + [("pfp", "trk", "len", "", "", "")]).pfp.trk.len.groupby(level=[0,1,2]).nth(-2).rename("subl_proton_length")
    slcdf = multicol_add(slcdf, subl_proton_length)

    bad_tmatch = np.invert(slcdf.slc.tmatch.eff > 0.5) & (slcdf.slc.tmatch.idx >= 0)
    slcdf.loc[bad_tmatch, ("slc","tmatch","idx")] = np.nan

    # match # of column levels
    mcdf.columns = pd.MultiIndex.from_tuples([tuple(list(c) +["", "", "", ""]) for c in mcdf.columns])

    print(slcdf.reset_index())
    print(mcdf.reset_index())

    df = pd.merge(slcdf.reset_index(), 
                mcdf.reset_index(),
                left_on=[("__ntuple", "", ""), ("entry", "", ""), ("slc", "tmatch", "idx")], 
                right_on=[("__ntuple", "", ""), ("entry", "", ""), ("rec.mc.nu..index", "", "")], 
                how="left"
                ) 

    df = df.set_index(slcdf.index.names, verify_integrity=True)

    return df



In [66]:
slc_trkdf = make_evtdf_sbnd(f)

    entry rec.slc..index rec.slc.reco.pfp..index             slc              \
                                                 is_clear_cosmic      vertex   
                                                                           x   
                                                                               
                                                                               
                                                                               
                                                                               
0       0              0                       0               0  -56.326027   
1       0              1                       0               0  107.954773   
2       1              0                       0               0   25.229219   
3       1              0                       1               0   25.229219   
4       2              0                       0               0 -148.088028   
..    ...            ...                

KeyError: ('__ntuple', '', '')

In [22]:
for k in f["recTree"].keys():
    if "is_neutrino" in k:
        print(k)

In [54]:
partdf = make_epartdf(f)

In [57]:
partdf
mudf = partdf[(partdf.pid == 2)].sort_values(partdf.index.names[:2] + [("length", "", "")]).groupby(level=[0,1]).last()

In [59]:
partdf

end_point              \
                                                        x           y   
                                                                        
entry rec.dlp..index rec.dlp.particles..index                           
0     0              0                               -inf        -inf   
                     1                               -inf        -inf   
                     2                               -inf        -inf   
                     3                         -63.840000  134.789993   
      1              0                               -inf        -inf   
...                                                   ...         ...   
10    32             2                               -inf        -inf   
                     3                         167.160004   77.190002   
      33             0                               -inf        -inf   
                     1                               -inf        -inf   
                     2                           7.860000 -181.110001   

                                                          is_contained  \
                                                        z                
                                                                         
entry rec.dlp..index rec.dlp.particles..index                            
0     0              0                               -inf            1   
                     1                               -inf            1   
                     2                               -inf            1   
                     3                         853.898987            0   
      1              0                               -inf            1   
...                                                   ...          ...   
10    32             2                               -inf            1   
                     3                         444.398987            0   
      33             0                               -inf            0   
                     1                               -inf            0   
                     2                         443.498993            0   

                                              is_primary is_principal_match  \
                                                                              
                                                                              
entry rec.dlp..index rec.dlp.particles..index                                 
0     0              0                                 1                  0   
                     1                                 0                  0   
                     2                                 0                  0   
                     3                                 1                  0   
      1              0                                 0                  0   
...                                                  ...                ...   
10    32             2                                 0                  0   
                     3                                 1                  0   
      33             0                                 0                  0   
                     1                                 0                  0   
                     2                                 1                  0   

                                              is_valid      length  \
                                                                     
                                                                     
entry rec.dlp..index rec.dlp.particles..index                        
0     0              0                               1   -1.000000   
                     1                               1   -1.000000   
                     2                               1   -1.000000   
                     3                               1  348.870819   
      1              0                               1   -1.000000   
...                      

In [58]:
mudf

end_point                         is_contained  \
                               x           y           z                
                                                                        
entry rec.dlp..index                                                    
0     0               -63.840000  134.789993  853.898987            0   
      1               -51.540001  134.190002 -866.301025            0   
      2              -199.440002  134.190002 -156.800995            0   
      3              -102.839996  109.290001 -117.801003            0   
      4              -181.440002  100.889999  582.999023            0   
...                          ...         ...         ...          ...   
10    29                7.860000   10.590000  171.998993            0   
      30              253.559998  -50.310001  208.598999            0   
      31              160.259995 -132.210007  314.498993            0   
      32              167.160004   77.190002  444.398987            0   
      33                7.860000 -181.110001  443.498993            0   

                     is_primary is_principal_match is_valid      length  \
                                                                          
                                                                          
entry rec.dlp..index                                                      
0     0                       1                  0        1  348.870819   
      1                       1                  0        1   70.764374   
      2                       1                  0        1  318.159912   
      3                       1                  0        1  313.743591   
      4                       1                  0        1  248.099792   
...                         ...                ...      ...         ...   
10    29                      1                  0        1  195.010483   
      30                      1                  0        1  131.689285   
      31                      1                  0        1    6.083501   
      32                      1                  0        1  131.844971   
      33                      1                  0        1  204.374908   

                         csda_ke            ke  ...       truth             \
                                                ...         end       genp   
                                                ...           z          x   
entry rec.dlp..index                            ...                          
0     0               789.733671  99999.997291  ...  783.314026  -1.701810   
      1               188.565487  99999.997354  ... -820.362305  -2.548505   
      2               720.972727   1568.955941  ... -141.486694   0.230568   
      3               711.132655   1889.815427  ...  -48.486355  -3.071355   
      4               566.600221   4713.849013  ...  564.296448  -1.754650   
...                          ...           ...  ...         ...        ...   
10    29              451.746340   3957.969941  ...  178.083115  -3.316366   
      30              317.118977  99999.997360  ...  543.519653  18.847168   
      31               34.951271     17.370777  ...  314.302277   1.751574   
      32              317.447076   2577.157117  ...  543.519653  18.847168   
      33              471.889718  99999.997378  ...  443.870270  -1.505084   

                                                                              \
                                               length   G4ID parent cont_tpc   
                             y          z                                      
entry rec.dlp..index                                                           
0     0              -4.335444  -1.011521  349.611298  293.0    0.0      0.0   
      1              -2.835912   0.381922  287.918945  237.0    0.0      0.0   
      2              -3.289594   0.080318  319.676758  401.0    0.0      0.0   
      3              -5.820331   1.628441  339.268829  449.0    0.0   